<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install surprise
!pip install apyori
!pip install matrix_factorization
!pip install factor-analyzer
!pip install python-helper

In [ ]:
#All the header files required for the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
from zipfile import ZipFile as zip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d rounakbanik/the-movies-dataset


 95% 217M/228M [00:05<00:00, 38.4MB/s]
100% 228M/228M [00:05<00:00, 45.2MB/s]


In [ ]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = '/content/the-movies-dataset.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [ ]:
# Apriori


# Data Preprocessing
dataset = pd.read_csv('movies_metadata.csv', header = None)
dataset.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
2,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
3,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
4,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34


In [ ]:
ratings_df = pd.read_csv("ratings_small.csv")



In [ ]:
movies_df=pd.read_csv("movies_metadata.csv")

In [ ]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movies_df.drop(movies_df.index[19730],inplace=True)
movies_df.drop(movies_df.index[29502],inplace=True)
movies_df.drop(movies_df.index[35585],inplace=True)

In [ ]:
#to resolve the error when we merge 
movies_df.id = movies_df.id.astype(np.int64)


In [ ]:
type(movies_df.id[0])


numpy.int64

In [ ]:
ratings_df.movieId.isin(movies_df.id).sum()
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
ratings_df = pd.merge(ratings_df,movies_df[['title','id']],left_on='movieId',right_on='id')
ratings_df.head()



,userId,movieId,rating,timestamp,title,id
0,1,1371,2.5,1260759135,Rocky III,1371
1,4,1371,4.0,949810302,Rocky III,1371
2,7,1371,3.0,851869160,Rocky III,1371
3,19,1371,4.0,855193404,Rocky III,1371
4,21,1371,3.0,853852263,Rocky III,1371


In [ ]:
ratings_df.drop(['timestamp','id'],axis=1,inplace=True)


In [ ]:
ratings_df["rating"].fillna(0)

0        2.5
1        4.0
2        3.0
3        4.0
4        3.0
        ... 
44989    4.0
44990    3.0
44991    4.0
44992    3.0
44993    3.0
Name: rating, Length: 44994, dtype: float64

In [ ]:
ratings_df.shape

(44994, 4)

In [ ]:
ratings_df.sample(5)


,userId,movieId,rating,title
14814,442,344,4.0,"Bang, Boom, Bang"
23307,572,145,4.5,Breaking the Waves
9885,199,858,3.0,Sleepless in Seattle
37250,285,2116,3.0,Out of Time
44927,621,4104,0.5,Benny & Joon


In [ ]:
ratings_df.isnull().sum()


userId     0
movieId    0
rating     0
title      0
dtype: int64

In [ ]:
ratings_count = ratings_df.groupby(by="title")['rating'].count().reset_index().rename(columns={'rating':'totalRatings'})[['title','totalRatings']]

In [ ]:
ratings_count.shape[0]



2794

In [ ]:
len(ratings_df['title'].unique())



2794

In [ ]:
ratings_count.sample(5)

,title,totalRatings
17,2 Days in Paris,12
2042,The Chronicles of Riddick,5
1148,Krull,28
1314,May Fools,5
555,Daisy Miller,3


In [ ]:
ratings_count.head()



,title,totalRatings
0,!Women Art Revolution,2
1,'Gator Bait,1
2,'Twas the Night Before Christmas,2
3,...And God Created Woman,1
4,00 Schneider - Jagd auf Nihil Baxter,2


In [ ]:
ratings_df.head()


,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,4,1371,4.0,Rocky III
2,7,1371,3.0,Rocky III
3,19,1371,4.0,Rocky III
4,21,1371,3.0,Rocky III


In [ ]:
ratings_total = pd.merge(ratings_df,ratings_count,on='title',how='left')

In [ ]:
ratings_total.shape



(44994, 5)

In [ ]:
ratings_total.head()



,userId,movieId,rating,title,totalRatings
0,1,1371,2.5,Rocky III,47
1,4,1371,4.0,Rocky III,47
2,7,1371,3.0,Rocky III,47
3,19,1371,4.0,Rocky III,47
4,21,1371,3.0,Rocky III,47


In [ ]:
ratings_count['totalRatings'].describe()



count    2794.000000
mean       16.103794
std        31.481306
min         1.000000
25%         1.000000
50%         4.000000
75%        15.750000
max       324.000000
Name: totalRatings, dtype: float64

In [ ]:
ratings_count['totalRatings'].quantile(np.arange(.6,1,0.01))



0.60      7.00
0.61      7.00
0.62      7.00
0.63      8.00
0.64      8.00
0.65      9.00
0.66      9.00
0.67     10.00
0.68     10.00
0.69     11.00
0.70     12.00
0.71     12.00
0.72     13.00
0.73     14.00
0.74     14.00
0.75     15.75
0.76     17.00
0.77     18.00
0.78     19.00
0.79     20.00
0.80     21.00
0.81     22.33
0.82     24.00
0.83     26.00
0.84     27.00
0.85     29.00
0.86     31.00
0.87     34.00
0.88     37.00
0.89     41.77
0.90     45.00
0.91     49.00
0.92     52.56
0.93     59.00
0.94     64.42
0.95     71.00
0.96     83.28
0.97     98.21
0.98    119.14
0.99    168.49
Name: totalRatings, dtype: float64

In [ ]:
votes_count_threshold = 20



In [ ]:
ratings_top = ratings_total.query('totalRatings > @votes_count_threshold')


In [ ]:
ratings_top.shape

(34552, 5)

In [ ]:
ratings_top.head()

,userId,movieId,rating,title,totalRatings
0,1,1371,2.5,Rocky III,47
1,4,1371,4.0,Rocky III,47
2,7,1371,3.0,Rocky III,47
3,19,1371,4.0,Rocky III,47
4,21,1371,3.0,Rocky III,47


In [ ]:
if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])

In [ ]:
ratings_top.shape

(34412, 5)

In [ ]:
df_for_knn = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0)

In [ ]:
df_for_knn.head()
df_for_knn.dropna()


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hour Party People,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
28 Days Later,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28 Weeks Later,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Adam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Frankenstein,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Young and Innocent,0.0,0.0,3.5,0.0,3.5,0.0,0.0,4.5,4.0,0.0,3.0,0.0,3.0,0.0,1.0,0.0,4.5,0.0,0.0,3.5,0.0,4.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [ ]:
df_for_knn.shape



(580, 671)

In [ ]:
from scipy.sparse import csr_matrix


In [ ]:
df_for_knn_sparse = csr_matrix(df_for_knn.values)
df_for_knn_sparse.T

<671x580 sparse matrix of type '<class 'numpy.float64'>'
	with 34412 stored elements in Compressed Sparse Column format>

In [ ]:
from sklearn.neighbors import NearestNeighbors



In [ ]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')



In [ ]:
model_knn.fit(df_for_knn_sparse)



NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
query_index = np.random.choice(df_for_knn.shape[0])

In [ ]:
distances, indices = model_knn.kneighbors(df_for_knn.loc['Batman Returns'].values.reshape(1,-1),n_neighbors=6)

In [ ]:
distances, indices = model_knn.kneighbors(df_for_knn.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [ ]:
for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendations for movie: {0}\n".format(df_for_knn.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}".format(i,df_for_knn.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for movie: Gleaming the Cube

1: Blood: The Last Vampire, with distance of 0.46415363025704304
2: One Night at McCool's, with distance of 0.48446876319387455
3: Dogtown and Z-Boys, with distance of 0.48733299540736397
4: Arlington Road, with distance of 0.4991811052471211
5: Whale Rider, with distance of 0.5075058991998447


In [ ]:
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

In [ ]:
df_for_ar = df_for_knn.T.applymap(encode_units)
df_for_ar.dropna()


title,"20,000 Leagues Under the Sea",2001: A Space Odyssey,24 Hour Party People,28 Days Later,28 Weeks Later,300,48 Hrs.,5 Card Stud,7 Virgins,8 Women,88 Minutes,A Bridge Too Far,A Brief History of Time,A Clockwork Orange,A Kiss Before Dying,A Nightmare on Elm Street,A Perfect Murder,A River Runs Through It,A Tale of Two Cities,A Time to Kill,A Very Long Engagement,A View to a Kill,"A Woman, a Gun and a Noodle Shop",Abraham,Addicted to Love,Adventures of Zatôichi,Aelita: Queen of Mars,Aguirre: The Wrath of God,Ali: Fear Eats the Soul,Alien,Aliens vs Predator: Requiem,All the King's Men,All the Way Boys,Almost Famous,American Beauty,American Graffiti,American Pie,American Pie 2,"Amigo, Stay Away",Amélie,...,"Torrente, the Dumb Arm of the Law",Totally Blonde,Tough Enough,True Romance,Tuya's Marriage,Twin Peaks: Fire Walk with Me,Two Brothers,Two Weeks Notice,Uncle Buck,Under the Sand,Underworld,Van Gogh,Voyage to the Bottom of the Sea,Wag the Dog,Waiter,Walk the Line,Wayne's World 2,We Own the Night,We're No Angels,Wedlock,Whale Rider,What's Eating Gilbert Grape,When Saturday Comes,While You Were Sleeping,Who Killed Bambi?,Will Penny,Willy Wonka & the Chocolate Factory,Windows on Monday,Wings of Desire,Wish You Were Here,Within the Woods,X-Men Origins: Wolverine,Y Tu Mamá También,Yankee Doodle Dandy,Yesterday,Young Adam,Young Frankenstein,Young and Innocent,Zatoichi,xXx
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,1,0,0,0,...,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,0.0,1,0,0,0.0,0,1,0.0,0,0,0,0.0,0.0,1.0,0.0,1.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0.0,1,0,0.0,0.0,0,0.0,0.0,0,0,0,1.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,1,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,0.0,0,0,0,0.0,1,0,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,1.0,0.0,0,0,0,0,0,0,...,0,0,0.0,0,1,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,1.0,0.0,0
4,1.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,1,0.0,0.0,0.0,0,0,1,1,0,0,...,0,0,1.0,0,0,0.0,0.0,0,1.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,1,1.0,0.0,0,0.0,0,0.0,0,1,0.0,0,0.0,0.0,1,0.0,0.0,0
5,0.0,0,0,0,0.0,0,1,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0.0,1.0,0.0,0,0,0,1,0,0,...,0,0,0.0,0,1,1.0,1.0,0,0.0,0.0,1,0,0,1.0,1,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0,0,0,0.0,0,1,0.0,0,0,0,0.0,0.0,1.0,0.0,1.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,1,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
668,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
669,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0


In [ ]:
df_for_ar.replace({False: 0, True: 1}, inplace=True)
df_for_ar.notnull()

df_for_ar.notnull().astype('int')
df_for_ar.fillna(0)
df_for_ar.fillna(0, inplace=True)   # Replace all NaN values with zero, updating the DataFrame directly

df_for_ar.head()



title,"20,000 Leagues Under the Sea",2001: A Space Odyssey,24 Hour Party People,28 Days Later,28 Weeks Later,300,48 Hrs.,5 Card Stud,7 Virgins,8 Women,88 Minutes,A Bridge Too Far,A Brief History of Time,A Clockwork Orange,A Kiss Before Dying,A Nightmare on Elm Street,A Perfect Murder,A River Runs Through It,A Tale of Two Cities,A Time to Kill,A Very Long Engagement,A View to a Kill,"A Woman, a Gun and a Noodle Shop",Abraham,Addicted to Love,Adventures of Zatôichi,Aelita: Queen of Mars,Aguirre: The Wrath of God,Ali: Fear Eats the Soul,Alien,Aliens vs Predator: Requiem,All the King's Men,All the Way Boys,Almost Famous,American Beauty,American Graffiti,American Pie,American Pie 2,"Amigo, Stay Away",Amélie,...,"Torrente, the Dumb Arm of the Law",Totally Blonde,Tough Enough,True Romance,Tuya's Marriage,Twin Peaks: Fire Walk with Me,Two Brothers,Two Weeks Notice,Uncle Buck,Under the Sand,Underworld,Van Gogh,Voyage to the Bottom of the Sea,Wag the Dog,Waiter,Walk the Line,Wayne's World 2,We Own the Night,We're No Angels,Wedlock,Whale Rider,What's Eating Gilbert Grape,When Saturday Comes,While You Were Sleeping,Who Killed Bambi?,Will Penny,Willy Wonka & the Chocolate Factory,Windows on Monday,Wings of Desire,Wish You Were Here,Within the Woods,X-Men Origins: Wolverine,Y Tu Mamá También,Yankee Doodle Dandy,Yesterday,Young Adam,Young Frankenstein,Young and Innocent,Zatoichi,xXx
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,1,0,0,0,...,0,0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,0.0,1,0,0,0.0,0,1,0.0,0,0,0,0.0,0.0,1.0,0.0,1.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0.0,1,0,0.0,0.0,0,0.0,0.0,0,0,0,1.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,1,0.0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,0.0,0,0,0,0.0,1,0,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,1.0,0.0,0,0,0,0,0,0,...,0,0,0.0,0,1,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,1.0,0.0,0
4,1.0,0,0,0,0.0,0,0,0.0,0,0,0,0.0,0.0,1.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,1,0.0,0.0,0.0,0,0,1,1,0,0,...,0,0,1.0,0,0,0.0,0.0,0,1.0,0.0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,1,1.0,0.0,0,0.0,0,0.0,0,1,0.0,0,0.0,0.0,1,0.0,0.0,0
5,0.0,0,0,0,0.0,0,1,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0.0,1.0,0.0,0,0,0,1,0,0,...,0,0,0.0,0,1,1.0,1.0,0,0.0,0.0,1,0,0,1.0,1,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0,1.0,0.0,0


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
frequent_itemsets = apriori(df_for_ar, min_support=0.07, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(48 Hrs.),"(20,000 Leagues Under the Sea)",0.298063,0.129657,0.076006,0.255000,1.966724,0.037360,1.168245
1,"(20,000 Leagues Under the Sea)",(48 Hrs.),0.129657,0.298063,0.076006,0.586207,1.966724,0.037360,1.696349
2,"(20,000 Leagues Under the Sea)",(A Nightmare on Elm Street),0.129657,0.266766,0.081967,0.632184,2.369807,0.047379,1.993480
3,(A Nightmare on Elm Street),"(20,000 Leagues Under the Sea)",0.266766,0.129657,0.081967,0.307263,2.369807,0.047379,1.256382
4,"(20,000 Leagues Under the Sea)",(Back to the Future Part II),0.129657,0.210134,0.077496,0.597701,2.844379,0.050251,1.963381
